# **SCRAPER**

## **Initializing the scraper**

In [ ]:
#: Import the libraries for the scraping
import undetected_chromedriver as uc  # type: ignore
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import os

#! FUNCTIONS:
#! Function to initialize the Chrome driver
def init_driver() -> uc.Chrome:
    options = uc.ChromeOptions()

    #options.add_argument('--headless')
    #options.add_argument('--no-sandbox')
    # options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('--disable-gpu')
    # options.add_argument('--disable-extensions')
    # options.add_argument('--blink-settings=imagesEnabled=false')
    
    driver = uc.Chrome(options=options)

    return driver

## **Scraping house url**

### List with all zones in Bilbao

In [ ]:
# The distrits of Bilbao are: Deusto, Uribarri, Otxarkoaga/Txurdinaga, 
#                              Basurto/Zorroza, Abando, Begoña, 
#                              Rekalde, e Ibaiondo 

zonas = {"deusto":[97, []], "uribarri":[136, []], "otxarkoaga-txurdinaga":[36, []],
         "basurto-zorroza":[151, []], "abando-albia":[275, []], "indautxu":[164, []], "casco-viejo":[65, []], 
         "begona-santutxu":[85, []], "rekalde":[213, []], "ibaiondo":[212, []], "san-adrian-la-pena":[44, []]}

### Scraping logic

In [ ]:
# Add checked areas beacause sometimes we are banned from the web page, so we don't start again from the beginning
zonas_miradas = ["deusto", "uribarri", "otxarkoaga-txurdinaga", "basurto-zorroza", "abando-albia", 
                 "indautxu", "casco-viejo", "rekalde", "ibaiondo", "san-adrian-la-pena"]

driver = init_driver()

for ubicacion in zonas:
    if ubicacion in zonas_miradas: continue
    for page in range(1, (zonas[ubicacion][0] // 30) + 2): # +2 porque +1 para que llegue al valor y +1 para que mire tambien el resto

        url_base = f"https://www.idealista.com/venta-viviendas/bilbao/{ubicacion}/pagina-{page}.htm"

        driver.get(url_base)

        sleep(10)

        # 1. Sacar enlaces de cada anuncio
        links = driver.find_elements(By.CSS_SELECTOR, "a.item-link")
        urls_detalle = [a.get_attribute("href") for a in links]

        zonas[ubicacion][1].extend(urls_detalle)
        
        sleep(10)


### Test completeness

In [ ]:
# Comprobar que tenemos todas las casa
for ubicacion in zonas:
    print(f"En la ubicacion: {ubicacion} hay {zonas[ubicacion][0]} casas y hay {len(zonas[ubicacion][1])} urls")

En la ubicacion: deusto hay 97 casas y hay 0 urls
En la ubicacion: uribarri hay 136 casas y hay 0 urls
En la ubicacion: otxarkoaga-txurdinaga hay 36 casas y hay 0 urls
En la ubicacion: basurto-zorroza hay 151 casas y hay 0 urls
En la ubicacion: abando-albia hay 275 casas y hay 0 urls
En la ubicacion: indautxu hay 164 casas y hay 0 urls
En la ubicacion: casco-viejo hay 65 casas y hay 0 urls
En la ubicacion: begona-santutxu hay 85 casas y hay 88 urls
En la ubicacion: rekalde hay 213 casas y hay 0 urls
En la ubicacion: ibaiondo hay 212 casas y hay 0 urls
En la ubicacion: san-adrian-la-pena hay 44 casas y hay 0 urls


### Safe data into CSV

In [ ]:
import csv

csv_file = "../data/raw/idealista_urls.csv"

with open(csv_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["zona", "url"])  # cabecera

    for zona, (n_total, urls) in zonas.items():
        for url in urls:
            writer.writerow([zona, url])

print(f"CSV generado: {csv_file}")


CSV generado: idealista_urls.csv


## **Scraping house details**

###  Reading from csv

In [ ]:
import csv

def csv_a_lista(ruta_csv):
    lista = []
    with open(ruta_csv, mode="r", encoding="utf-8") as f:
        reader = csv.reader(f)
        for fila in reader:
            if fila:  # evitar líneas vacías
                lista.append([fila[0], fila[1]])
    return lista

# Ejemplo de uso
lista_zona_url = csv_a_lista("../data/raw/idealista_urls.csv")

print(lista_zona_url[:5])  # muestra las primeras 5 filas

[['deusto', 'https://www.idealista.com/inmueble/109356873/'], ['deusto', 'https://www.idealista.com/inmueble/106221410/'], ['deusto', 'https://www.idealista.com/inmueble/107750109/'], ['deusto', 'https://www.idealista.com/inmueble/106221526/'], ['deusto', 'https://www.idealista.com/inmueble/108491309/']]


### Sraper Class

In [5]:
from bs4 import BeautifulSoup
import re
import json
from typing import Dict, Optional

class PropertyScraper:
    def __init__(self):
        pass
    
    def extract_property_data_from_driver(self, driver) -> Dict:
        """
        Extrae toda la información de una propiedad desde el driver de Selenium
        """
        html_content = driver.page_source
        return self.extract_property_data(html_content)

    def extract_property_data(self, html_content: str) -> Dict:
        """f
        Extrae toda la información de una propiedad desde el HTML
        """
        soup = BeautifulSoup(html_content, 'html.parser')
        
        property_data = {
            'url': None,
            'precio': None,
            'zona': None,
            'barrio': None,
            'm2_construidos': None,
            'm2_utiles': None,
            'n_habitaciones': None,
            'n_baños': None,
            'n_planta': None,
            'exterior_interior': None,
            'ascensor': None,
            'garaje': None,
            'trastero': None,
            'balcon': None,
            'obra_nueva_segunda_mano': None,
            'estado': None,
            'año_construccion': None,
            'profesional': None,
            'calefaccion': None,
            'consumo_valor': None,
            'consumo_etiqueta': None,
            'emisiones_valor': None,
            'emisiones_etiqueta': None,
            'descripcion': None
        }

        # Extraer barrio
        property_data['barrio'] = self._extract_neighborhood(soup)
        
        # Extraer descripción
        property_data['descripcion'] = self._extract_description(soup)
        
        # Extraer precios
        property_data['precio'] = self._extract_price(soup)

        # Extraer profesional/inmobiliaria
        property_data['profesional'] = self._extract_professional(soup)
        
        # Extraer información de .info-features
        self._extract_info_features(soup, property_data)

        # Extraer número de planta desde la referencia (si existe)
        self._extract_floor_from_reference(soup, property_data)
        
        # Extraer información detallada de las listas
        self._extract_detailed_features(soup, property_data)
        
        # Extraer certificado energético
        self._extract_energy_certificate(soup, property_data)
        
        return property_data
    
    def _extract_professional(self, soup: BeautifulSoup) -> Optional[str]:
        """Extrae el nombre del profesional/inmobiliaria"""
        professional_div = soup.find('div', class_='professional-name')
        if professional_div:
            tipo_profesiona_div = soup.find('div', class_= 'name').get_text(strip=True)
            # Buscar el span que contiene el nombre
            span = professional_div.find('span')
            if span and tipo_profesiona_div != "Particular":
                # Extraer el texto directamente del span (sin los inputs)
                name = span.get_text(strip=True)
                # Limpiar espacios múltiples
                name = re.sub(r'\s+', ' ', name)
                return name if name else None
        return None
    
    def _extract_description(self, soup: BeautifulSoup) -> Optional[str]:
        """Extrae la descripción de la propiedad"""
        comment_div = soup.find('div', class_='comment')
        if comment_div:
            # Buscar el div con la clase adCommentsLanguage que contiene el párrafo
            ad_comment = comment_div.find('div', class_='adCommentsLanguage')
            if ad_comment:
                # Extraer el texto del párrafo
                p_element = ad_comment.find('p')
                if p_element:
                    # Obtener el texto y reemplazar <br> por espacios
                    description = p_element.get_text(separator=' ', strip=True)
                    # Limpiar espacios múltiples
                    description = re.sub(r'\s+', ' ', description)
                    return description
        return None
    
    def _extract_neighborhood(self, soup: BeautifulSoup) -> Optional[str]:
        """Extrae el barrio de la propiedad"""
        neighborhood_element = soup.find('span', class_='main-info__title-minor')
        if neighborhood_element:
            return neighborhood_element.get_text(strip=True)
        return None
    
    def _extract_price(self, soup: BeautifulSoup) -> Optional[str]:
        """Extrae el precio de la propiedad"""
        price_element = soup.find('span', class_='info-data-price')
        if price_element:
            price_text = price_element.get_text(strip=True)
            # Extraer solo los números y puntos del precio
            price_match = re.search(r'([\d.]+)', price_text.replace('€', ''))
            return price_match.group(1) if price_match else None
        return None
    
    def _extract_floor_from_reference(self, soup: BeautifulSoup, data: Dict):
        """Extrae el número de planta desde la referencia del anuncio"""
        # Solo extraer si aún no tenemos el número de planta
        if data['n_planta'] is not None:
            return
        
        ad_reference_div = soup.find('div', class_='ad-reference-container')
        if ad_reference_div:
            txt_ref = ad_reference_div.find('p', class_='txt-ref')
            if txt_ref:
                ref_text = txt_ref.get_text(strip=True)
                # Buscar patrón como "2 1ºA" donde el número antes de º es la planta
                floor_match = re.search(r'(\d+)º', ref_text)
                if floor_match:
                    data['n_planta'] = floor_match.group(1)
    
    def _extract_info_features(self, soup: BeautifulSoup, data: Dict):
        """Extrae información básica del div info-features"""
        info_features = soup.find('div', class_='info-features')
        if not info_features:
            return
        
        spans = info_features.find_all('span')
        
        for span in spans:
            text = span.get_text(strip=True)
            
            # Obra nueva
            if 'Obra nueva' in text:
                data['obra_nueva_segunda_mano'] = 'Obra nueva'
            
            # Metros cuadrados
            if 'm²' in text:
                m2_match = re.search(r'(\d+)\s*m²', text)
                if m2_match and not data['m2_construidos']:
                    data['m2_construidos'] = m2_match.group(1)
            
            # Habitaciones
            if 'hab.' in text:
                hab_match = re.search(r'(\d+)\s*hab\.', text)
                if hab_match:
                    data['n_habitaciones'] = hab_match.group(1)
            
            # Planta y características del edificio
            if 'Planta' in text:
                # Extraer número de planta
                planta_match = re.search(r'Planta\s+(\d+)ª', text)
                if planta_match:
                    data['n_planta'] = planta_match.group(1)
                
                # Exterior/Interior
                if 'exterior' in text.lower():
                    data['exterior_interior'] = 'exterior'
                elif 'interior' in text.lower():
                    data['exterior_interior'] = 'interior'
                
                # Ascensor
                if 'con ascensor' in text.lower():
                    data['ascensor'] = 'Sí'
                elif 'sin ascensor' in text.lower():
                    data['ascensor'] = 'No'
            
            # Garaje
            if 'Garaje incluido' in text:
                data['garaje'] = 'Incluido'
    
    def _extract_detailed_features(self, soup: BeautifulSoup, data: Dict):
        """Extrae información detallada de las listas ul"""
        # Buscar todos los divs con clase 'details-property'
        details_divs = soup.find_all('div', class_='details-property')
        
        if not details_divs:
            return
        
        # Filtrar para obtener solo el div correcto (el que tiene las características)
        # Evitar el div con clase 'profile-qualification-homes-info'
        target_div = None
        for div in details_divs:
            # Verificar que no tenga la clase 'profile-qualification-homes-info'
            if 'profile-qualification-homes-info' not in div.get('class', []):
                target_div = div
                break
        
        if not target_div:
            return
        
        # Buscar todas las listas dentro del div correcto
        ul_elements = target_div.find_all('ul')
        
        for ul in ul_elements:
            li_elements = ul.find_all('li')
            
            for li in li_elements:
                text = li.get_text(strip=True)
                self._parse_feature_text(text, data)
    
    def _parse_feature_text(self, text: str, data: Dict):
        """Analiza el texto de cada característica y extrae la información"""
        text_lower = text.lower()
        
        # Metros cuadrados construidos/útiles
        if 'm²' in text:
            # Buscar patrón: "67 m² construidos, 65 m² útiles"
            construidos_match = re.search(r'(\d+)\s*m²\s+construidos', text)
            utiles_match = re.search(r'(\d+)\s*m²\s+útiles', text)
            
            if construidos_match:
                data['m2_construidos'] = construidos_match.group(1)
            if utiles_match:
                data['m2_utiles'] = utiles_match.group(1)
            
            # Si no hay especificación "construidos" o "útiles", asumir construidos
            if not construidos_match and not utiles_match and not data['m2_construidos']:
                m2_match = re.search(r'(\d+)\s*m²', text)
                if m2_match:
                    data['m2_construidos'] = m2_match.group(1)
        
        # Habitaciones
        if 'habitaciones' in text_lower or 'habitación' in text_lower:
            hab_match = re.search(r'(\d+)', text)
            if hab_match:
                data['n_habitaciones'] = hab_match.group(1)
        
        # Baños
        if 'baños' in text_lower or 'baño' in text_lower:
            baño_match = re.search(r'(\d+)', text)
            if baño_match:
                data['n_baños'] = baño_match.group(1)
        
        # Balcón/Terraza
        if 'balcón' in text_lower:
            data['balcon'] = 'Sí'
        elif 'terraza' in text_lower:
            if not data['balcon']:
                data['balcon'] = 'Terraza'
            else:
                data['balcon'] = 'Balcón y terraza'
        
        # Garaje
        if 'plaza de garaje' in text_lower or 'garaje' in text_lower:
            if 'incluida' in text_lower or 'incluido' in text_lower:
                data['garaje'] = 'Incluido'
            else:
                data['garaje'] = 'Sí'
        
        # Estado de la vivienda
        if 'segunda mano' in text_lower:
            data['obra_nueva_segunda_mano'] = 'Segunda mano'
            if 'buen estado' in text_lower:
                data['estado'] = 'Buen estado'
            elif 'reformar' in text_lower:
                data['estado'] = 'A reformar'
        elif 'obra nueva' in text_lower or 'promoción de obra nueva' in text_lower:
            data['obra_nueva_segunda_mano'] = 'Obra nueva'
        
        # Trastero
        if 'trastero' in text_lower:
            data['trastero'] = 'Sí'

        # Calefacción (solo individual o central)
        if 'calefacción' in text_lower or 'calefaccion' in text_lower:
            if 'individual' in text_lower:
                data['calefaccion'] = 'Individual'
            elif 'central' in text_lower:
                data['calefaccion'] = 'Central'
        
        # Año de construcción
        if 'construido en' in text_lower:
            año_match = re.search(r'(\d{4})', text)
            if año_match:
                data['año_construccion'] = año_match.group(1)
        
        # Planta (desde edificio)
        if 'planta' in text_lower and 'ª' in text:
            planta_match = re.search(r'(\d+)ª', text)
            if planta_match:
                data['n_planta'] = planta_match.group(1)
            
            if 'exterior' in text_lower:
                data['exterior_interior'] = 'exterior'
            elif 'interior' in text_lower:
                data['exterior_interior'] = 'interior'
        
        # Ascensor
        if 'con ascensor' in text_lower:
            data['ascensor'] = 'Sí'
        elif 'sin ascensor' in text_lower:
            data['ascensor'] = 'No'
    
    def _extract_energy_certificate(self, soup: BeautifulSoup, data: Dict):
        """Extrae información del certificado energético"""
        # Buscar la sección del certificado energético
        energy_section = soup.find('h2', string=re.compile(r'Certificado energético'))
        if not energy_section:
            return
        
        # Buscar el div padre que contiene la información energética
        energy_div = energy_section.find_next('div', class_='details-property_features')
        if not energy_div:
            return
        
        ul_element = energy_div.find('ul')
        if not ul_element:
            return
        
        li_elements = ul_element.find_all('li')
        
        for li in li_elements:
            text = li.get_text(strip=True)
            
            # Consumo
            if 'Consumo:' in text:
                consumo_span = li.find('span', class_=re.compile(r'icon-energy-c-'))
                if consumo_span:
                    consumo_text = consumo_span.get_text(strip=True)
                    if consumo_text:
                        # Extraer valor numérico
                        valor_match = re.search(r'([\d,]+(?:\.\d+)?)', consumo_text)
                        if valor_match:
                            data['consumo_valor'] = valor_match.group(1)
                    
                    # Extraer etiqueta energética de la clase
                    class_list = consumo_span.get('class', [])
                    for class_name in class_list:
                        if 'icon-energy-c-' in class_name:
                            etiqueta = class_name.split('-')[-1].upper()
                            data['consumo_etiqueta'] = etiqueta
            
            # Emisiones
            elif 'Emisiones:' in text:
                emisiones_span = li.find('span', class_=re.compile(r'icon-energy-c-'))
                if emisiones_span:
                    emisiones_text = emisiones_span.get_text(strip=True)
                    if emisiones_text:
                        # Extraer valor numérico
                        valor_match = re.search(r'([\d,]+(?:\.\d+)?)', emisiones_text)
                        if valor_match:
                            data['emisiones_valor'] = valor_match.group(1)
                    
                    # Extraer etiqueta energética de la clase
                    class_list = emisiones_span.get('class', [])
                    for class_name in class_list:
                        if 'icon-energy-c-' in class_name:
                            etiqueta = class_name.split('-')[-1].upper()
                            data['emisiones_etiqueta'] = etiqueta
    

    #! FUNCION QUE EMPIEZA EL CICLO DEL SCRAPING (SIN GUARDAR EN CSV)
    def scrape_properties_from_zone_urls(self, driver, zone_url_pairs: list) -> list:
        """
        Hace scraping de múltiples URLs con información de zona usando el driver de Selenium
        
        Args:
            driver: Instancia del driver de Selenium ya inicializada
            zone_url_pairs: Lista de listas en formato [[zona1, url1], [zona1, url2], [zona2, url3], ...]
            
        Returns:
            Lista de diccionarios con los datos de cada propiedad incluyendo la zona
        """
        results = []
        
        for i, (zona, url) in enumerate(zone_url_pairs):
            try:
                print(f"Procesando propiedad {i+1}/{len(zone_url_pairs)} - Zona: {zona} - URL: {url}")
                
                # Navegar a la URL
                driver.get(url)
                
                # Esperar un poco para que cargue la página (opcional)
                sleep(10)
                
                # Extraer datos
                property_data = self.extract_property_data_from_driver(driver)
                property_data['zona'] = zona  # Añadir la zona recibida por parámetro
                property_data['url'] = url    # Añadir la URL para referencia
                
                results.append(property_data)
                
            except Exception as e:
                print(f"Error procesando {url} (Zona: {zona}): {e}")
                # Añadir entrada vacía para mantener el orden
                results.append({
                    'url': None,
                    'precio': None,
                    'zona': None,
                    'barrio': None,
                    'm2_construidos': None,
                    'm2_utiles': None,
                    'n_habitaciones': None,
                    'n_baños': None,
                    'n_planta': None,
                    'exterior_interior': None,
                    'ascensor': None,
                    'garaje': None,
                    'trastero': None,
                    'balcon': None,
                    'obra_nueva_segunda_mano': None,
                    'estado': None,
                    'año_construccion': None,
                    'profesional': None,
                    'calefaccion': None,
                    'consumo_valor': None,
                    'consumo_etiqueta': None,
                    'emisiones_valor': None,
                    'emisiones_etiqueta': None,
                    'descripcion': None,
                    'error': str(e)
                    })
        return results
    

    #! FUNCION QUE EMPIEZA EL CICLO DEL SCRAPING (GUARDANDO EN CSV)
    def scrape_properties_from_zone_urls_with_csv(self, driver, zone_url_pairs: list, csv_filename: str = 'propiedades_scraping.csv') -> list:
        """
        Hace scraping de múltiples URLs guardando cada propiedad inmediatamente en CSV
        
        Args:
            driver: Instancia del driver de Selenium ya inicializada
            zone_url_pairs: Lista de listas en formato [[zona1, url1], [zona1, url2], [zona2, url3], ...]
            csv_filename: Nombre del archivo CSV donde guardar los datos
            
        Returns:
            Lista de diccionarios con los datos de cada propiedad incluyendo la zona
        """
        results = []
        
        # Definir las columnas del CSV
        fieldnames = [
            'url', 'precio', 'zona', 'barrio',
            'm2_construidos', 'm2_utiles',
            'n_habitaciones', 'n_baños', 'n_planta',
            'exterior_interior', 'ascensor', 'garaje',
            'trastero', 'balcon', 'obra_nueva_segunda_mano', 
            'estado', 'año_construccion', 'calefaccion', 
            'profesional', 'consumo_valor', 'consumo_etiqueta',
            'emisiones_valor', 'emisiones_etiqueta',
            'descripcion', 'error'
        ]

        
        # Verificar si el archivo existe para saber si escribir headers
        file_exists = os.path.isfile(csv_filename)
        
        # Abrir el archivo en modo append
        with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Escribir headers solo si el archivo es nuevo
            if not file_exists:
                writer.writeheader()
            
            for i, (zona, url) in enumerate(zone_url_pairs):
                try:
                    print(f"Procesando propiedad {i+1}/{len(zone_url_pairs)} - Zona: {zona} - URL: {url}")
                    
                    # Navegar a la URL
                    driver.get(url)
                    
                    # Esperar un poco para que cargue la página
                    import time
                    time.sleep(10)
                    
                    # Extraer datos
                    property_data = self.extract_property_data_from_driver(driver)
                    property_data['url'] = url
                    property_data['zona'] = zona
                    property_data['error'] = None
                    
                    # Guardar inmediatamente en CSV
                    writer.writerow(property_data)
                    csvfile.flush()  # Forzar escritura al disco
                    
                    results.append(property_data)
                    print(f"✓ Guardada en CSV: {url}")
                    
                except Exception as e:
                    print(f"✗ Error procesando {url} (Zona: {zona}): {e}")
                    
                    # Guardar el error también en CSV
                    error_data = {
                        'url': None,
                        'precio': None,
                        'zona': None,
                        'barrio': None,
                        'm2_construidos': None,
                        'm2_utiles': None,
                        'n_habitaciones': None,
                        'n_baños': None,
                        'n_planta': None,
                        'exterior_interior': None,
                        'ascensor': None,
                        'garaje': None,
                        'trastero': None,
                        'balcon': None,
                        'obra_nueva_segunda_mano': None,
                        'estado': None,
                        'año_construccion': None,
                        'profesional': None,
                        'calefaccion': None,
                        'consumo_valor': None,
                        'consumo_etiqueta': None,
                        'emisiones_valor': None,
                        'emisiones_etiqueta': None,
                        'descripcion': None,
                        'error': str(e)
                    }
                    
                    writer.writerow(error_data)
                    csvfile.flush()
                    results.append(error_data)
        
        print(f"\n✓ Scraping completado. Datos guardados en: {csv_filename}")
        return results

### Functions to start scraping

In [7]:
# Funcion que crea el objeto PropertyScraper y hace que empiece el scraping
def scrape_properties_with_zones_selenium(driver, zone_url_pairs):
    """
    Función para hacer scraping de propiedades con información de zona usando Selenium
    
    Args:
        driver: Driver de Selenium inicializado
        zone_url_pairs: Lista de listas en formato [[zona1, url1], [zona1, url2], [zona2, url3], ...]
    
    Returns:
        Lista de diccionarios con todos los datos extraídos incluyendo zona
    """
    scraper = PropertyScraper()
    return scraper.scrape_properties_from_zone_urls(driver, zone_url_pairs)

# Funcion que hace lo mismo que la anterios pero guarda datos en csv
def scrape_properties_with_zones_selenium_to_csv(driver, zone_url_pairs, csv_filename='propiedades_scraping.csv'):
    """
    Función para hacer scraping de propiedades guardando cada una inmediatamente en CSV
    
    Args:
        driver: Driver de Selenium inicializado
        zone_url_pairs: Lista de listas en formato [[zona1, url1], [zona1, url2], [zona2, url3], ...]
        csv_filename: Nombre del archivo CSV (por defecto 'propiedades_scraping.csv')
    
    Returns:
        Lista de diccionarios con todos los datos extraídos
    """
    scraper = PropertyScraper()
    return scraper.scrape_properties_from_zone_urls_with_csv(driver, zone_url_pairs, csv_filename)

### Scraping main

In [ ]:
# Ejemplo de uso con Selenium:
if __name__ == "__main__":
    
    # Tu driver ya inicializado
    driver = init_driver()  # o el que uses
    
    
    #? Comentamos este metodo porque no guarda la informacion en csv
    #all_data = scrape_properties_with_zones_selenium(driver, lista_zona_url)

    # Scraping con zonas y guardando en csv
    all_data = scrape_properties_with_zones_selenium_to_csv(driver, lista_zona_url, '../data/raw/idealista_viviendas_detalle.csv')

    # Guardar resultados
    # import pandas as pd
    # df = pd.DataFrame(all_data)
    # df.to_csv('propiedades_scrapeadas.csv', index=False)
    
    driver.quit()

    pass